In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# === Load data ===
df = pd.read_csv("touch_force_dataset.csv")
X = df.iloc[:, :-3].values
y = df.iloc[:, -3:].values

# === Split and scale ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# === Convert to tensors ===
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(-1)  # (N, 208, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(-1)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

X_train_scaled = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# === Transformer Model ===
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # (1, max_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerRegressor(nn.Module):
    def __init__(self, seq_len=208, d_model=64, nhead=8, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(1, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=128, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):  # x: (B, 208, 1)
        x = self.input_proj(x)           # (B, 208, d_model)
        x = self.pos_encoder(x)          # (B, 208, d_model)
        x = self.transformer(x)          # (B, 208, d_model)
        x = x.mean(dim=1)                # Global average pooling
        return self.regressor(x)         # (B, 3)

model = TransformerRegressor().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()

# === Training loop ===
epochs = 1000
batch_size = 64
print("🔧 Training Transformer Regressor...")

for epoch in range(epochs):
    model.train()
    perm = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        idx = perm[i:i+batch_size]
        x_batch = X_train_tensor[idx].to(device)
        y_batch = y_train_tensor[idx].to(device)

        optimizer.zero_grad()
        y_pred = model(x_batch).to(device)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 50 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.6f}")

# === Evaluation ===
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()

y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_y.inverse_transform(y_test_tensor.cpu().numpy())

# === Metrics ===
mae = mean_absolute_error(y_test_unscaled, y_pred, multioutput="raw_values")
r2 = r2_score(y_test_unscaled, y_pred, multioutput="raw_values")

print("\n📊 Transformer Model Performance:")
print(f"MAE - X:     {mae[0]:.4f}")
print(f"MAE - Y:     {mae[1]:.4f}")
print(f"MAE - Force: {mae[2]:.4f}")
print(f"R²  - X:     {r2[0]:.4f}")
print(f"R²  - Y:     {r2[1]:.4f}")
print(f"R²  - Force: {r2[2]:.4f}")

# === Plot predictions ===
labels = ["x", "y", "force"]
for i in range(3):
    plt.figure(figsize=(4.5, 4))
    plt.scatter(y_test_unscaled[:, i], y_pred[:, i], alpha=0.7)
    plt.xlabel(f"True {labels[i]}")
    plt.ylabel(f"Predicted {labels[i]}")
    plt.title(f"Transformer Prediction: {labels[i]}")
    plt.grid(True)
    plt.axis("equal")
    plt.plot([min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
             [min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
             'r--')
    plt.tight_layout()
    plt.savefig(f"transformer_pred_{labels[i]}.png", dpi=300)
    plt.close()

print("✅ Training complete. Plots saved in 'data/'")


🔧 Training Transformer Regressor...
Epoch   0 | Loss: 0.809660
Epoch  50 | Loss: 0.025465
Epoch 100 | Loss: 0.039520
Epoch 150 | Loss: 0.032006
Epoch 200 | Loss: 0.027571
Epoch 250 | Loss: 0.020881
Epoch 300 | Loss: 0.061110
Epoch 350 | Loss: 0.010580
Epoch 400 | Loss: 0.019286
Epoch 450 | Loss: 0.026459
Epoch 500 | Loss: 0.009672
Epoch 550 | Loss: 0.013310
Epoch 600 | Loss: 0.004204
Epoch 650 | Loss: 0.060574
Epoch 700 | Loss: 0.012968
Epoch 750 | Loss: 0.007417
Epoch 800 | Loss: 0.030547
Epoch 850 | Loss: 0.006928
Epoch 900 | Loss: 0.009704
Epoch 950 | Loss: 0.008481
Epoch 999 | Loss: 0.004332

📊 Transformer Model Performance:
MAE - X:     0.0203
MAE - Y:     0.0191
MAE - Force: 0.0516
R²  - X:     0.9937
R²  - Y:     0.9951
R²  - Force: 0.9152
✅ Training complete. Plots saved in 'data/'


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# === Set output folder ===
os.makedirs("outputs", exist_ok=True)

# === GPU/CPU setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# === Load dataset ===
df = pd.read_csv("touch_force_dataset.csv")
X = df.iloc[:, :-3].values
y = df.iloc[:, -3:].values

# === Train/test split and normalization ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# === Tensor conversion ===
X_train_tensor = torch.tensor(X_train_scaled[:, np.newaxis, :], dtype=torch.float32).to(device)  # (N, 1, 208)
X_test_tensor = torch.tensor(X_test_scaled[:, np.newaxis, :], dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

# === Positional Encoding ===
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

# === CNN + Transformer hybrid ===
class CNNTransformerRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),  # output: (32, 104)
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2)   # output: (64, 52)
        )
        self.proj = nn.Linear(64, 64)
        self.pos_encoder = PositionalEncoding(d_model=64, max_len=52)

        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=8, dim_feedforward=128, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.regressor = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):  # x: (B, 1, 208)
        x = self.cnn(x)              # (B, 64, 52)
        x = x.permute(0, 2, 1)       # (B, 52, 64)
        x = self.proj(x)             # ensure model dim matches
        x = self.pos_encoder(x)
        x = self.transformer(x)      # (B, 52, 64)
        x = x.mean(dim=1)            # Global average pooling
        return self.regressor(x)

# === Initialize ===
model = CNNTransformerRegressor().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()

# === Training loop ===
epochs = 500
batch_size = 64
print("🔧 Training CNN + Transformer Model...")

for epoch in range(epochs):
    model.train()
    perm = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        idx = perm[i:i+batch_size]
        x_batch = X_train_tensor[idx]
        y_batch = y_train_tensor[idx]

        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 50 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.6f}")

# === Evaluation ===
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()

y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_y.inverse_transform(y_test_tensor.cpu().numpy())

# === Metrics ===
mae = mean_absolute_error(y_test_unscaled, y_pred, multioutput="raw_values")
r2 = r2_score(y_test_unscaled, y_pred, multioutput="raw_values")

print("\n📊 CNN + Transformer Performance:")
print(f"MAE - X:     {mae[0]:.4f}")
print(f"MAE - Y:     {mae[1]:.4f}")
print(f"MAE - Force: {mae[2]:.4f}")
print(f"R²  - X:     {r2[0]:.4f}")
print(f"R²  - Y:     {r2[1]:.4f}")
print(f"R²  - Force: {r2[2]:.4f}")

# === Plot ===
labels = ["x", "y", "force"]
for i in range(3):
    plt.figure(figsize=(4.5, 4))
    plt.scatter(y_test_unscaled[:, i], y_pred[:, i], alpha=0.7)
    plt.xlabel(f"True {labels[i]}")
    plt.ylabel(f"Predicted {labels[i]}")
    plt.title(f"Hybrid Model Prediction: {labels[i]}")
    plt.grid(True)
    plt.axis("equal")
    plt.plot([min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
             [min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
             'r--')
    plt.tight_layout()
    plt.savefig(f"outputs/cnn_transformer_{labels[i]}.png", dpi=300)
    plt.close()

print("\n✅ Training complete. Prediction plots saved to '/content/outputs/'")


✅ Using device: cuda
🔧 Training CNN + Transformer Model...
Epoch   0 | Loss: 0.918251
Epoch  50 | Loss: 0.041723
Epoch 100 | Loss: 0.024288
Epoch 150 | Loss: 0.031773
Epoch 200 | Loss: 0.007061
Epoch 250 | Loss: 0.017170
Epoch 300 | Loss: 0.010626
Epoch 350 | Loss: 0.014407
Epoch 400 | Loss: 0.004340
Epoch 450 | Loss: 0.008311
Epoch 499 | Loss: 0.005354

📊 CNN + Transformer Performance:
MAE - X:     0.0159
MAE - Y:     0.0143
MAE - Force: 0.0526
R²  - X:     0.9968
R²  - Y:     0.9974
R²  - Force: 0.9095

✅ Training complete. Prediction plots saved to '/content/outputs/'


##Physics-informed CNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# === Output path ===
os.makedirs("outputs", exist_ok=True)

# === Device setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# === Load data ===
df = pd.read_csv("touch_force_dataset.csv")
X = df.iloc[:, :-3].values
y = df.iloc[:, -3:].values

# === Train/test split and normalization ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# === Tensor conversion ===
X_train_tensor = torch.tensor(X_train_scaled[:, np.newaxis, :], dtype=torch.float32).to(device)  # (N, 1, 208)
X_test_tensor = torch.tensor(X_test_scaled[:, np.newaxis, :], dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

# === CNN Model ===
class CNNRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),  # (B, 32, 104)
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),  # (B, 64, 52)
            nn.Flatten(),     # (B, 64*52)
            nn.Linear(64 * 52, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.model(x)

# === Initialize ===
model = CNNRegressor().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()
lambda_phys = 0.01  # weight of physics-informed loss

# === Training ===
epochs = 700
batch_size = 64
print("🔧 Training Physics-Informed CNN...")

for epoch in range(epochs):
    model.train()
    perm = torch.randperm(X_train_tensor.size(0))

    for i in range(0, X_train_tensor.size(0), batch_size):
        idx = perm[i:i + batch_size]
        x_batch = X_train_tensor[idx]
        y_batch = y_train_tensor[idx]

        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss_pred = loss_fn(y_pred, y_batch)

        # === Physics-informed loss (monotonicity) ===
        v_diff_norm = torch.norm(x_batch.squeeze(1), dim=1)  # raw ΔV magnitude
        force_pred = y_pred[:, 2]
        # Normalize both
        v_diff_norm = (v_diff_norm - v_diff_norm.mean()) / v_diff_norm.std()
        force_pred_norm = (force_pred - force_pred.mean()) / force_pred.std()

        loss_phys = loss_fn(v_diff_norm, force_pred_norm)

        total_loss = loss_pred + lambda_phys * loss_phys
        total_loss.backward()
        optimizer.step()

    if epoch % 50 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch:3d} | Pred Loss: {loss_pred.item():.6f} | Phys Loss: {loss_phys.item():.6f}")

# === Evaluation ===
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()

y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_y.inverse_transform(y_test_tensor.cpu().numpy())

# === Metrics ===
mae = mean_absolute_error(y_test_unscaled, y_pred, multioutput="raw_values")
r2 = r2_score(y_test_unscaled, y_pred, multioutput="raw_values")

print("\n📊 Physics-Informed CNN Performance:")
print(f"MAE - X:     {mae[0]:.4f}")
print(f"MAE - Y:     {mae[1]:.4f}")
print(f"MAE - Force: {mae[2]:.4f}")
print(f"R²  - X:     {r2[0]:.4f}")
print(f"R²  - Y:     {r2[1]:.4f}")
print(f"R²  - Force: {r2[2]:.4f}")

# === Plot predictions ===
labels = ["x", "y", "force"]
for i in range(3):
    plt.figure(figsize=(4.5, 4))
    plt.scatter(y_test_unscaled[:, i], y_pred[:, i], alpha=0.7)
    plt.xlabel(f"True {labels[i]}")
    plt.ylabel(f"Predicted {labels[i]}")
    plt.title(f"Physics-Informed CNN Prediction: {labels[i]}")
    plt.grid(True)
    plt.axis("equal")
    plt.plot(
        [min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
        [min(y_test_unscaled[:, i]), max(y_test_unscaled[:, i])],
        "r--",
    )
    plt.tight_layout()
    plt.savefig(f"outputs/phys_cnn_pred_{labels[i]}.png", dpi=300)
    plt.close()

print("✅ Training complete. Plots saved in 'outputs/'")


✅ Using device: cuda
🔧 Training Physics-Informed CNN...
Epoch   0 | Pred Loss: 0.206990 | Phys Loss: 0.699358
Epoch  50 | Pred Loss: 0.032033 | Phys Loss: 0.884131
Epoch 100 | Pred Loss: 0.016652 | Phys Loss: 1.200459
Epoch 150 | Pred Loss: 0.004111 | Phys Loss: 1.470896
Epoch 200 | Pred Loss: 0.011548 | Phys Loss: 0.587657
Epoch 250 | Pred Loss: 0.008346 | Phys Loss: 0.954195
Epoch 300 | Pred Loss: 0.002399 | Phys Loss: 0.706706
Epoch 350 | Pred Loss: 0.002182 | Phys Loss: 0.446137
Epoch 400 | Pred Loss: 0.005578 | Phys Loss: 0.641890
Epoch 450 | Pred Loss: 0.004999 | Phys Loss: 0.587430
Epoch 500 | Pred Loss: 0.001362 | Phys Loss: 1.512943
Epoch 550 | Pred Loss: 0.002285 | Phys Loss: 1.105213
Epoch 600 | Pred Loss: 0.010008 | Phys Loss: 0.722931
Epoch 650 | Pred Loss: 0.008182 | Phys Loss: 0.548184
Epoch 699 | Pred Loss: 0.002204 | Phys Loss: 0.942559

📊 Physics-Informed CNN Performance:
MAE - X:     0.0086
MAE - Y:     0.0096
MAE - Force: 0.0442
R²  - X:     0.9988
R²  - Y:     0.99